In [28]:
import json 
import pandas as pd
import time
import tiktoken
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist


In [52]:
data = pd.read_json('restrauntData.json')

def get_text(x):
    review_list = []
    for i in x:
        review_list.append(i["text"])
    return review_list

data["review_list"] = data["reviews"].apply(get_text)
data["latitude"] = data["location"].apply(lambda x: x["lat"])
data["longitude"] = data["location"].apply(lambda x: x["lng"])

In [53]:
def join_text(dfx):
    return_str = ''
    return_str = return_str.join(filter(None,dfx))
    return return_str

string_reviews = pd.DataFrame()

In [54]:
def summarize_reviews(rev_list):
    # Combine all reviews into a single string
    all_reviews = "".join(rev_list)

    # Tokenize the string into sentences and words
    sentences = sent_tokenize(all_reviews)
    words = word_tokenize(all_reviews)

    # Create a frequency distribution of words
    freq_dist = FreqDist(words)

    # Create a list of the most common words
    common_words = [word for word, freq in freq_dist.most_common(100)]

    # Create a summary by selecting the most relevant sentences
    summary_sentences = []
    for sentence in sentences:
        sentence_words = word_tokenize(sentence)
        relevance = sum(1 for word in sentence_words if word in common_words)
        if relevance > 0:
            summary_sentences.append((sentence, relevance))

    # Sort the summary sentences by relevance
    summary_sentences.sort(key=lambda x: x[1], reverse=True)

    # Return the first n sentences as the summary
    summary = [sentence for sentence, relevance in summary_sentences[:2]]
    return "".join(summary)


In [55]:
string_reviews["str_rev"] =  data["reviews"].copy()

string_reviews["str_rev"] =  data["review_list"].apply(join_text)

In [56]:
string_reviews["str_rev"] 

0      Out of everything?  How...Rude employees, they...
1      Not open during it's peak hours, doesn't follo...
2      Very nice bloody Mary’s and good service from ...
3      Very bland. Over cooked. High priced and NASTY...
4      The staff, the ambience, the drinks, and the d...
                             ...                        
295    Just don't do it. There are plenty of restaura...
296    Unbelievably delicious chicken!!! Very fair pr...
297    Loved this spot ! Food was so fresh . Handmade...
298    Kelly is fantastic.Gyro was good,a little.dry ...
299    Amazing place to have african foods With an aw...
Name: str_rev, Length: 300, dtype: object

In [57]:
string_reviews['review_summaries'] = string_reviews['str_rev'].apply(summarize_reviews)

In [58]:
string_reviews

,str_rev,review_summaries
0,"Out of everything? How...Rude employees, they...",I waited and then recieved salted... then some...
1,"Not open during it's peak hours, doesn't follo...",I think the hardest part is the waiting list w...
2,Very nice bloody Mary’s and good service from ...,Anyway she gave y our order to a couple that w...
3,Very bland. Over cooked. High priced and NASTY...,My pepperoni pizza wasnt even cooked all the w...
4,"The staff, the ambience, the drinks, and the d...",But fundamentally if your steak or chicken jus...
...,...,...
295,Just don't do it. There are plenty of restaura...,I’m not as familiar with Chicago-style dogs (I...
296,Unbelievably delicious chicken!!! Very fair pr...,"O well, smoothie was good, for an airport loca..."
297,Loved this spot ! Food was so fresh . Handmade...,!If your hungry for some amazing food then thi...
298,"Kelly is fantastic.Gyro was good,a little.dry ...",Lots of regulars and frequently visited by the...
